<a href="https://colab.research.google.com/github/francoriboratig/TP2-Datos/blob/master/Red%20convolucional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import numpy as np
import tensorflow as tf
import pandas as pd
import random

!pip install -q tensorflow-hub
!pip install -q tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")


Version:  2.2.0
Eager mode:  True
Hub version:  0.8.0
GPU is NOT AVAILABLE


In [53]:
url_train = 'https://raw.githubusercontent.com/JulioCastillo1/TP1-Datos/master/train.csv'

train_df = pd.read_csv(url_train)
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [54]:
train_df = train_df.drop(['keyword', 'location'], axis = 1)

In [55]:
train_df.head()

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1


In [56]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

train_text_list = np.asarray(train_df['text'])
train_target_list = np.asarray(train_df['target'])

hub_layer(train_text_list[:3])



<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 0.00723878,  0.15689617,  1.2953067 , -0.04077986, -0.16036902,
        -1.2004429 , -0.40451345, -0.5239726 , -0.23558664, -0.35022184,
        -0.28988177, -0.47197863, -0.07621863,  0.17048942, -0.8323773 ,
         0.02260084,  0.16125458, -0.64086777, -0.569074  ,  0.04000134],
       [-0.4983734 ,  1.2828879 , -0.18836124, -0.94734734,  0.8805808 ,
        -0.17885518, -0.01543166,  0.3832704 , -1.508848  ,  1.7532811 ,
        -0.75493276,  0.60124093, -0.14378418, -0.891944  ,  0.43213663,
         1.3467408 , -1.4859253 ,  1.5762157 , -0.37878054,  0.9423684 ],
       [ 0.08814443, -0.31140247,  0.6044149 ,  1.7130805 ,  1.3203121 ,
        -3.201303  ,  0.8445131 ,  0.9621795 , -1.5353138 ,  2.2069528 ,
        -0.2790253 ,  0.01095965, -2.2282734 ,  0.2043116 , -2.17501   ,
         1.3441613 ,  0.11884003, -1.2388006 , -0.25501075,  0.69527215]],
      dtype=float32)>

In [57]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_7 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_14 (Dense)             (None, 16)                336       
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [58]:
model.compile(optimizer = 'adam',
              loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics = ['accuracy'])

# LO QUE SIGUE NO ANDA, no tengo train_data (ni validation_data, pero es análogo), deben
# tener formato Prefetch DataShape y no se cómo crearlo a partir de train_text_list, train_df o lo que sea

history = model.fit(train_text_list, train_target_list, batch_size=512,
                    epochs=20,
                    verbose=1)


Epoch 1/20
15/15 [==============================] - 0s 11ms/step - loss: 0.6728 - accuracy: 0.5765
Epoch 2/20
15/15 [==============================] - 0s 10ms/step - loss: 0.6145 - accuracy: 0.6149
Epoch 3/20
15/15 [==============================] - 0s 12ms/step - loss: 0.5767 - accuracy: 0.6612
Epoch 4/20
15/15 [==============================] - 0s 11ms/step - loss: 0.5491 - accuracy: 0.6951
Epoch 5/20
15/15 [==============================] - 0s 10ms/step - loss: 0.5282 - accuracy: 0.7172
Epoch 6/20
15/15 [==============================] - 0s 11ms/step - loss: 0.5105 - accuracy: 0.7358
Epoch 7/20
15/15 [==============================] - 0s 11ms/step - loss: 0.4955 - accuracy: 0.7544
Epoch 8/20
15/15 [==============================] - 0s 11ms/step - loss: 0.4824 - accuracy: 0.7647
Epoch 9/20
15/15 [==============================] - 0s 11ms/step - loss: 0.4697 - accuracy: 0.7734
Epoch 10/20
15/15 [==============================] - 0s 11ms/step - loss: 0.4581 - accuracy: 0.7826
Epoch 11/

In [63]:
_,precision = model.evaluate(train_text_list, train_target_list)
print('Precisión: %.2f' % (precision*100))

238/238 [==============================] - 1s 2ms/step - loss: 0.3487 - accuracy: 0.8467
Precisión: 84.67
